In [24]:
import threading
import time
import sqlite3

# Define global mutex locks
mutex_lock = threading.Lock()
read_count_lock = threading.Lock()
read_count = 0

# Define a worker thread that updates the database
def writer_thread(order):
    # Acquire the mutex lock to ensure exclusive access to the database
    mutex_lock.acquire()

    # Update the database with the new order
    conn = sqlite3.connect('orders.db')
    c = conn.cursor()
    c.execute('INSERT INTO orders VALUES (?)', (order,))
    conn.commit()
    conn.close()

    # Release the mutex lock
    mutex_lock.release()

# Define a reader thread that reads from the database
def reader_thread(order):
    global read_count

    # Acquire the read_count_lock
    read_count_lock.acquire()
    read_count += 1

    if read_count == 1:
        # Acquire the mutex lock if this is the first reader
        mutex_lock.acquire()

    # Release the read_count_lock
    read_count_lock.release()

    # Read the database
    conn = sqlite3.connect('orders.db')
    c = conn.cursor()
    c.execute('SELECT (?) FROM orders', (order,))
    print(f'Reader thread - Orders: {c.fetchall()} \n')
    conn.close()

    # Acquire the read_count_lock
    read_count_lock.acquire()
    read_count -= 1

    if read_count == 0:
        # Release the mutex lock if this is the last reader
        mutex_lock.release()

    # Release the read_count_lock
    read_count_lock.release()

# Define a function that handles incoming orders
def handle_order(order):
    # Create a new worker thread to handle the order
    writer = threading.Thread(target=writer_thread, args=(order,))
    writer.start()
    print(f'Order No. {order} received \n')

    # Continue handling incoming orders
    reader = threading.Thread(target=reader_thread, args=(order,))
    reader.start()

In [25]:
# Initialize the database
conn = sqlite3.connect('orders.db')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS orders')
c.execute('CREATE TABLE IF NOT EXISTS orders (order_id INTEGER)')
conn.commit()
conn.close()

# Start handling incoming orders and reading from the database
handle_order(1)
handle_order(2)
handle_order(3)

# Wait for the worker threads to finish
time.sleep(1)

Order No. 1 received 

Order No. 2 received 

Order No. 3 received 

Reader thread - Orders: [(1,)] 

Reader thread - Orders: [(2,)] 

Reader thread - Orders: [(3,)] 

